 ## Portfolio Optimization



## Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)


d = prices.shape[0] # number of months
n = prices.shape[1] # number of stocks

# monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


## Model 1
Four-asset portfolio: We can only invest in Microsoft (MSFT), Goldman Sachs (GS), Proctor & Gamble (PG), and U.S. Treasury Bonds (SCHP). Need to construct a minimum-variance portfolio with an expected monthly return of at least 0.5%.

\begin{align}
\text{Decision Variable}: w_{i}= \text{weight of stock i in the portfolio, i=1,2,3,4}
\end{align}

\begin{align}
\underset{{\bf w}}{\text{min}} \;\; &\sum_{i=1}^4\sum_{j=1}^4w_{i}w_{j}C_{ij}\\
&\text{s.t.}\\
&\sum_{i=1}^4{w_{i}r_{i}} \geq 0.5\\\
&\sum_{i=1}^4{w_{i}} = 1\\
&w_{i} \geq 0, \text{for  $i=1,2,3,4$}
\end{align}

In [2]:
#Intializing model 1
mod_1 = Model()

# decision variable
w = mod_1.addVars(390)

# defining the constraints 
cons_1 = mod_1.addConstr(sum(w[i] * avg_return[i] for i in tickind) >= 0.005)
cons_2 = mod_1.addConstr(sum(w[i] for i in tickind)== 1)

for i in tickind:
    mod_1.addConstr(w[i]>= 0.0)

# objective function 
mod_1.setObjective(sum(w[i]*w[j]*C[i,j] for i in tickind for j in tickind), GRB.MINIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-11


In [3]:
mod_1.update()
mod_1.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.1.0 23B81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 390 columns and 12 nonzeros
Model fingerprint: 0x24944e33
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 386 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.75648617e+03 -7.75648617e+03 

In [4]:
opt_risk = mod_1.objval
print(opt_risk)

0.00017749326422824248


In [5]:
opt_w = [w[i].x for i in tickind]
print(opt_w)

[0.23711755540145213, 0.025859609077561872, 1.6430790119996093e-10, 0.737022835356677]


In [6]:
import pandas as pd
opt_result=pd.DataFrame({'ticker':tick4, 'weight':opt_w})
opt_result_string = opt_result.to_string(index=False)



Model 1 Insights: 
- the optimal risk (i.e. the optimal objective function value): 1.77493264e-04
- solver time: 0.02 seconds
- the weight on each of the four stocks is as below

In [7]:
print(opt_result_string)

ticker       weight
  MSFT 2.371176e-01
    GS 2.585961e-02
    PG 1.643079e-10
  SCHP 7.370228e-01


### Model 2
Now we can invest in all 390 stocks. Construct a minimum-variance portfolio with an expected monthly return of at least 0.5%.

\begin{align}
\text{Decision Variable}: w_{i}= \text{weight of stock i in the portfolio, i=1,2,...,390}
\end{align}

\begin{align}
\underset{{\bf w}}{\text{min}} \;\; &\sum_{i=1}^{390}\sum_{j=1}^{390}w_{i}w_{j}C_{ij}\\
&\text{s.t.}\\
&\sum_{i=1}^{390}{w_{i}r_{i}} \geq 0.5\\\
&\sum_{i=1}^{390}{w_{i}} = 1\\
&w_{i} \geq 0, \text{for  $i=1,2,...,390$}
\end{align}

In [8]:
mod_2 = Model()

# defining the decision variable
w = mod_2.addVars(390)

# defining the constraints 
cons_1 = mod_2.addConstr(sum(w[i]*avg_return[i] for i in range(390)) >= 0.005)
cons_2 = mod_2.addConstr(sum(w[i] for i in range(390))== 1)

for i in range(390):
    mod_2.addConstr(w[i]>= 0.0)

# objective function 
mod_2.setObjective(sum(w[i]*w[j]*C[i,j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

In [9]:
mod_2.update()
mod_2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.1.0 23B81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0xd3a53ccf
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633e-17 -

In [10]:
opt_risk = mod_2.objval
print(opt_risk)

2.879175251400085e-05


Model 2 Insights:  
- the optimal risk: 2.87917525e-05
- solver time: 0.04 seconds

## Model 3
In practice, there are transaction fees associated with buying stocks. One way of keeping transaction fees low while still attaining desirable performance is to limit the total number of stocks that are purchased (i.e. limit the number of stocks that have a strictly positive weight). Construct a minimum-variance portfolio that selects at most 4 of the 390 stocks, and has an expected monthly return of at least 0.5%.

\begin{align}
\text{Decision Variable}: w_{i}= \text{weight of stock i in the portfolio, i=1,2,...,390}\\
\ x_{i}= \text{1 if selecte stock i, 0 if not (binary), i=1,2,...,390}
\end{align}

\begin{align}
\underset{{\bf w, x}}{\text{min}} \;\; &\sum_{i=1}^{390}\sum_{j=1}^{390}w_{i}w_{j}C_{ij}\\
&\text{s.t.}\\
&\sum_{i=1}^{390}{w_{i}r_{i}} \geq 0.5\\
&\sum_{i=1}^{390}{w_{i}} = 1\\
&\sum_{i=1}^{390}{x_{i}} \leq 4\\
&w_{i} \leq x_{i}, \text{for $i=1,2,...,390$}\\
&w_{i} \geq 0, \text{for $i=1,2,...,390$}
\end{align}

In [11]:
mod_3 = Model()

# defining the decision variable
w = mod_3.addVars(390)
x = mod_3.addVars(390, vtype = GRB.BINARY)

#defining the constraints 
cons_1 = mod_3.addConstr(sum(w[i]*avg_return[i] for i in range(390)) >= 0.005)
cons_2 = mod_3.addConstr(sum(w[i] for i in range(390)) == 1)
cons_3 = mod_3.addConstr(sum(x[i] for i in range(390)) <= 4)
cons_4 = mod_3.addConstrs(w[i]<= x[i] for i in range(390)) 
for i in range(390):
    mod_3.addConstr(w[i]>= 0.0)

# objective function 
mod_3.setObjective(sum(w[i]*w[j]*C[i,j] for i in range(390) for j in range(390)), GRB.MINIMIZE)

In [12]:
mod_3.update()
mod_3.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.1.0 23B81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x7dfc432c
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

In [13]:
opt_risk = mod_3.objval
print(opt_risk)

6.753470760728129e-05


In [14]:
opt_w = [w[i].x for i in range(390) if w[i].x>0]
opt_i = [i for i in range(390) if w[i].x>0]

# retrieve index that corresponds to each ticker
opt_tickers = [tickers[i] for i in opt_i] 

print(opt_i)
print(opt_tickers)
print(opt_w)

[118, 285, 348, 389]
['CME', 'LLY', 'NVDA', 'BND']
[0.12641141929490635, 0.0754761903543743, 0.04375370672843142, 0.7543586836222896]


In [15]:
opt_result3=pd.DataFrame({'index':opt_i, 'ticker':opt_tickers, 'weight':opt_w})
opt_result3_string = opt_result3.to_string(index=False)
print(opt_result3_string)

 index ticker   weight
   118    CME 0.126411
   285    LLY 0.075476
   348   NVDA 0.043754
   389    BND 0.754359


Model 3 Insights:
- the optimal risk: 6.753470760728e-05
- solver time: 15.23 seconds
- tikers & weights selected by the model are as below:

In [16]:
print(opt_result3_string)

 index ticker   weight
   118    CME 0.126411
   285    LLY 0.075476
   348   NVDA 0.043754
   389    BND 0.754359


#### Overall Insights 

> The optimal risk in Model 1 is higher than Model 2. This is primarily because we make the assumptions that all assets are risky assets, thus when we add more assets to our portfolio (as in model 2), the overall investment opportunity set becomes bigger (unless the new asset is redundant) and the efficient frontier shifts to the left. As a result, we are able to achieve the same expected return in model 2 but with a lower risk. So diversification helps us to mitigate the risks involved in model 2 as opposed to model 1 that has less number of risky assets.


> The optimal risk in Model 2 is lower than Model 3. This is using the same rationale as in part a). In this case, we are limiting our number of risky assets to a total of 4 and we are only considering those assets that have a strictly postitive weight. As a result, the efficient frontier for model 2 is to the left of the efficient frontier of model 3, which implies that we can achieve the same expected return from both the models, but with a lower risk in case of model 2 as opposed to less assets in model 3 where the risk will be higher.